## LangChain - Advanced SQL Queries in Natural Language

In [ ]:
_ = !pip install langchain

In [ ]:
import os
import sqlite3
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain

### Create and Populate Database

![image](2023-05-19_schema.png)

In [ ]:
conn = sqlite3.connect('charging_stations.sqlite')
cur = conn.cursor()

statements = [
    """
        CREATE TABLE IF NOT EXISTS city (
            id INTEGER PRIMARY KEY AUTOINCREMENT, 
            name VARCHAR
        );
    """,
    """
        CREATE TABLE IF NOT EXISTS charger_type (
            id INTEGER PRIMARY KEY AUTOINCREMENT, 
            name VARCHAR
        );
    """,
    """
        CREATE TABLE IF NOT EXISTS station (
            id INTEGER PRIMARY KEY AUTOINCREMENT, 
            city_id INTEGER,
            charger_type_id INT,
            FOREIGN KEY(city_id) REFERENCES city(id),
            FOREIGN KEY(charger_type_id) REFERENCES charger_type(id)
        );
    """,
    """
        CREATE TABLE IF NOT EXISTS available (
            station_id INTEGER, 
            available INTEGER,
            FOREIGN KEY(station_id) REFERENCES station(id)
        );
    """
]

for statement in statements:
    cur.execute(statement)
conn.commit()

In [ ]:
statements = [
    """INSERT INTO city (id, name) VALUES (1, 'Hong Kong');""",
    """INSERT INTO city (id, name) VALUES (2, 'Mumbai');""",
    """INSERT INTO city (id, name) VALUES (3, 'New York City');""",
    """INSERT INTO city (id, name) VALUES (4, 'Mexico City');""",
    """INSERT INTO city (id, name) VALUES (5, 'Teheran');""",
    
    
    """INSERT INTO charger_type (id, name) VALUES (1, 'AC');""",
    """INSERT INTO charger_type (id, name) VALUES (2, 'DC');""",
    
    
    """INSERT INTO station (id, city_id, charger_type_id) VALUES (1, 1, 1);""",
    """INSERT INTO station (id, city_id, charger_type_id) VALUES (2, 1, 1);""",
    """INSERT INTO station (id, city_id, charger_type_id) VALUES (3, 1, 1);""",
    """INSERT INTO station (id, city_id, charger_type_id) VALUES (4, 1, 2);""",
    
    """INSERT INTO station (id, city_id, charger_type_id) VALUES (5, 2, 1);""",
    """INSERT INTO station (id, city_id, charger_type_id) VALUES (6, 2, 1);""",
    """INSERT INTO station (id, city_id, charger_type_id) VALUES (7, 2, 1);""",
    """INSERT INTO station (id, city_id, charger_type_id) VALUES (8, 2, 1);""",
    """INSERT INTO station (id, city_id, charger_type_id) VALUES (9, 2, 1);""",
    """INSERT INTO station (id, city_id, charger_type_id) VALUES (11, 2, 1);""",
    """INSERT INTO station (id, city_id, charger_type_id) VALUES (12, 2, 1);""",
    """INSERT INTO station (id, city_id, charger_type_id) VALUES (13, 2, 1);""",
    """INSERT INTO station (id, city_id, charger_type_id) VALUES (14, 2, 1);""",
    
    """INSERT INTO station (id, city_id, charger_type_id) VALUES (15, 3, 2);""",
    
    """INSERT INTO station (id, city_id, charger_type_id) VALUES (16, 4, 1);""",
    """INSERT INTO station (id, city_id, charger_type_id) VALUES (17, 4, 1);""",
    """INSERT INTO station (id, city_id, charger_type_id) VALUES (18, 4, 1);""",
    """INSERT INTO station (id, city_id, charger_type_id) VALUES (19, 4, 2);""",
    """INSERT INTO station (id, city_id, charger_type_id) VALUES (20, 4, 2);""",
    """INSERT INTO station (id, city_id, charger_type_id) VALUES (21, 4, 2);""",
    
    """INSERT INTO station (id, city_id, charger_type_id) VALUES (22, 5, 1);""",
    """INSERT INTO station (id, city_id, charger_type_id) VALUES (23, 5, 1);""",
    """INSERT INTO station (id, city_id, charger_type_id) VALUES (24, 5, 1);""",
    """INSERT INTO station (id, city_id, charger_type_id) VALUES (25, 5, 2);""",
    
    
    """INSERT INTO available (station_id, available) VALUES (1, 0);""",
    """INSERT INTO available (station_id, available) VALUES (2, 1);""",
    """INSERT INTO available (station_id, available) VALUES (3, 0);""",
    """INSERT INTO available (station_id, available) VALUES (4, 1);""",
    """INSERT INTO available (station_id, available) VALUES (5, 1);""",
    """INSERT INTO available (station_id, available) VALUES (6, 0);""",
    """INSERT INTO available (station_id, available) VALUES (7, 1);""",
    """INSERT INTO available (station_id, available) VALUES (8, 0);""",
    """INSERT INTO available (station_id, available) VALUES (9, 1);""",
    """INSERT INTO available (station_id, available) VALUES (10, 1);""",
    """INSERT INTO available (station_id, available) VALUES (11, 1);""",
    """INSERT INTO available (station_id, available) VALUES (12, 0);""",
    """INSERT INTO available (station_id, available) VALUES (13, 0);""",
    """INSERT INTO available (station_id, available) VALUES (14, 0);""",
    """INSERT INTO available (station_id, available) VALUES (15, 1);""",
    """INSERT INTO available (station_id, available) VALUES (16, 1);""",
    """INSERT INTO available (station_id, available) VALUES (17, 0);""",
    """INSERT INTO available (station_id, available) VALUES (18, 1);""",
    """INSERT INTO available (station_id, available) VALUES (19, 0);""",
    """INSERT INTO available (station_id, available) VALUES (20, 0);""",
    """INSERT INTO available (station_id, available) VALUES (21, 1);""",
    """INSERT INTO available (station_id, available) VALUES (22, 1);""",
    """INSERT INTO available (station_id, available) VALUES (23, 1);""",
    """INSERT INTO available (station_id, available) VALUES (24, 1);""",
    """INSERT INTO available (station_id, available) VALUES (25, 0);""",
]
for statement in statements:
    cur.execute(statement)
conn.commit()

### Connect to OpenAI via LangChain

In [ ]:
db = SQLDatabase.from_uri('sqlite:///charging_stations.sqlite')
os.environ['OPENAI_API_KEY'] = '<your OpenAI API key here>'
llm = OpenAI(temperature=0)

In [ ]:
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

In [ ]:
prompt = """
How many stations are there in New York City?
"""
db_chain.run(prompt)

In [ ]:
prompt = """
How many available stations are there in Teheran?
"""
db_chain.run(prompt)

In [ ]:
prompt = """
A charger type 'DC' is needed to fast charge a car. 
Is there a station in Hong Kong that can be used to fast charge a car?
"""
db_chain.run(prompt)

In [ ]:
prompt = """
A charger type 'DC' is needed to fast charge a car.
The field available has to be 1 in order for the station to be available.
How many available stations are there in Mexico City that can be used to fast charge a car?
"""
db_chain.run(prompt)

**Disclaimer**:

Note: OpenAI provides a free API key for initial testing. Once you move to a paid subscription, calling the API in the way demonstrated in this example will incur monetary charges. Refer to OpenAI's pricing information for details.

Be aware that information, such as files to train OpenAI's LLM can become public if applied in the way this demo demonstrates. Refer to OpenAI's usage policy for details.